# Topic identification for a fake news Identifier

This project was originally from [here](https://www.datacamp.com/courses/natural-language-processing-fundamentals-in-python).

## Basic Concepts

Before analyzing our fake news dataset, I will quickly describe a few important concepts that will be used.

### Word Vectors

From [Wikipedia](https://en.wikipedia.org/wiki/Word_embedding):

> Word embedding is the collective name for a set of language modeling and feature learning techniques in natural language processing (NLP) where words or phrases from the vocabulary are mapped to vectors of real numbers.

Word vectors are multi-dimensional representation of word which allows one to obtain relationships between words. These relationships are obtained by NLP algorithms based on how the words are used throughout a text corpus. An example is the difference between word vectors. [The difference is similar](https://www.datacamp.com/courses/natural-language-processing-fundamentals-in-python) between words such as man and women and kind and queen.


### `Gensim` dictionary class and corpus

This can be best explained with an example. Consider the list `quotes` containing quotes from the Chinese philosopher and writer [Lao Tzu](https://en.wikipedia.org/wiki/Laozi):
- `word_tokenize ` tokenizes the strings `quotes` (after converting tokens to lowercases and dropping stopwords)
- The `Dictionary` class creates a mapping with an id for each token which can be seen using `token2id`. 
- A `Gensim` corpus transforms a document into a bag-of-words using the tokens ids and also their frequency in the document. The corpus is a list of sublists, each sublist corresponding to one document

Since we will be counting tokens I introduced some extra repeated words in the quotes!

In [58]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # see the value of multiple statements at once.

In [16]:
#nltk.download('punkt')

In [74]:
import nltk
from gensim.corpora.dictionary import Dictionary
from nltk.tokenize import word_tokenize



quotes = ['When I let go of what I am am, I become become become what what I might be',
          'Mastering others is strength strength strength. Mastering Mastering Mastering Mastering Mastering\
          yourself yourself yourself yourself is true power',
          'When you are content content content to be simply yourself and do not compare or compete, everybody will respect you',
          'Great acts are made up of small deeds deeds deeds deeds',
          'An ant on the move does more than a dozing dozing dozing dozing ox',
          'Anticipate Anticipate Anticipate Anticipate the difficult by managing the easy',
          'Nature does not hurry, yet everything everything everything everything is accomplished accomplished accomplished',
          'To see things in the seed, that is genius genius genius genius genius genius']

tokenized_quotes = [word_tokenize(quote.lower()) for quote in quotes] 

In [75]:
from stop_words import get_stop_words
en_stop_words = get_stop_words('en')
tokenized_quotes_stopped = []
for token_lst in tokenized_quotes:
    tokenized_quotes_stopped.append([i for i in token_lst if not i in en_stop_words if len(i) > 4])
print(tokenized_quotes_stopped)

[['become', 'become', 'become', 'might'], ['mastering', 'others', 'strength', 'strength', 'strength', 'mastering', 'mastering', 'mastering', 'mastering', 'mastering', 'power'], ['content', 'content', 'content', 'simply', 'compare', 'compete', 'everybody', 'respect'], ['great', 'small', 'deeds', 'deeds', 'deeds', 'deeds'], ['dozing', 'dozing', 'dozing', 'dozing'], ['anticipate', 'anticipate', 'anticipate', 'anticipate', 'difficult', 'managing'], ['nature', 'hurry', 'everything', 'everything', 'everything', 'everything', 'accomplished', 'accomplished', 'accomplished'], ['things', 'genius', 'genius', 'genius', 'genius', 'genius', 'genius']]


In [76]:
dictionary = Dictionary(tokenized_quotes_stopped) 
print(dictionary.token2id)

{'become': 0, 'might': 1, 'mastering': 2, 'others': 3, 'power': 4, 'strength': 5, 'compare': 6, 'compete': 7, 'content': 8, 'everybody': 9, 'respect': 10, 'simply': 11, 'deeds': 12, 'great': 13, 'small': 14, 'dozing': 15, 'anticipate': 16, 'difficult': 17, 'managing': 18, 'accomplished': 19, 'everything': 20, 'hurry': 21, 'nature': 22, 'genius': 23, 'things': 24}


In [77]:
corpus = [dictionary.doc2bow(doc) for doc in tokenized_quotes_stopped]
print(corpus,'\n')
print(corpus[0])
print(tokenized_quotes_stopped[0])
print(corpus[1])
print(tokenized_quotes_stopped[1],'\n')
print(dictionary.token2id.keys())

[[(0, 3), (1, 1)], [(2, 6), (3, 1), (4, 1), (5, 3)], [(6, 1), (7, 1), (8, 3), (9, 1), (10, 1), (11, 1)], [(12, 4), (13, 1), (14, 1)], [(15, 4)], [(16, 4), (17, 1), (18, 1)], [(19, 3), (20, 4), (21, 1), (22, 1)], [(23, 6), (24, 1)]] 

[(0, 3), (1, 1)]
['become', 'become', 'become', 'might']
[(2, 6), (3, 1), (4, 1), (5, 3)]
['mastering', 'others', 'strength', 'strength', 'strength', 'mastering', 'mastering', 'mastering', 'mastering', 'mastering', 'power'] 

dict_keys(['become', 'might', 'mastering', 'others', 'power', 'strength', 'compare', 'compete', 'content', 'everybody', 'respect', 'simply', 'deeds', 'great', 'small', 'dozing', 'anticipate', 'difficult', 'managing', 'accomplished', 'everything', 'hurry', 'nature', 'genius', 'things'])


In the corpus above consider the first and second lists corresponding to the first and second quotes:

    corpus[0] -> [(0, 3), (1, 1)]
    corpus[1] -> [(2, 6), (3, 1), (4, 1), (5, 3)]

These tuples represent:

    (token id from the dictionary, token frequency in the quote)

The third tuple (2, 6) of corpus[1] e.g. says that the token 'mastering' with id = 2 (which can be obtained using `.get( )`) from the dictionary appeared six times in corpus[1]. 

In [78]:
dictionary.token2id.get("mastering")

2

### Most common terms

To obtain the most common terms in the second quote (and across all quotes) we can proceed as follows. First we sort the tuples in `corpus[1]` by frequency. Note the syntax here. The key defines the sorting criterion which is the function:

    w[x] = element of w with index x
    
This function is implemented using **lambda**. 

In [79]:
quote = corpus[1]
quote = sorted(quote, key=lambda w: w[1], reverse=True)
quote

[(2, 6), (5, 3), (3, 1), (4, 1)]

Using `dictionary.get(word_id)` we find the words corresponding to the id `word_id` in the dictionary. The `for` below identifies the frequency each term appears in the first quote of the corpus:

In [81]:
for word_id, word_count in quote[:3]:
    print(word_id,'|', word_count,'|',dictionary.get(word_id),'|', word_count)

2 | 6 | mastering | 6
5 | 3 | strength | 3
3 | 1 | others | 1


We now create an empty dictionary using `defaultdict` to include a total word count:

In [87]:
from collections import defaultdict
total_word_count = defaultdict(int)
total_word_count 

defaultdict(int, {})

We will also need `itertools.chain.from_iterable`. From the docs:

> Make an iterator that returns elements from the first iterable until it is exhausted, then proceeds to the next iterable, until all of the iterables are exhausted. Used for treating consecutive sequences as a single sequence. 

An example:

In [118]:
import itertools
import operator

print(list(itertools.chain.from_iterable(corpus[0:3])))

i=0
for word_id, word_count in itertools.chain.from_iterable(corpus[0:3]):
    print('(word_id, word_count) from quote {}:'.format(i),(word_id, word_count))
    i +=1

[(0, 3), (1, 1), (2, 6), (3, 1), (4, 1), (5, 3), (6, 1), (7, 1), (8, 3), (9, 1), (10, 1), (11, 1)]
(word_id, word_count) from quote 0: (0, 3)
(word_id, word_count) from quote 1: (1, 1)
(word_id, word_count) from quote 2: (2, 6)
(word_id, word_count) from quote 3: (3, 1)
(word_id, word_count) from quote 4: (4, 1)
(word_id, word_count) from quote 5: (5, 3)
(word_id, word_count) from quote 6: (6, 1)
(word_id, word_count) from quote 7: (7, 1)
(word_id, word_count) from quote 8: (8, 3)
(word_id, word_count) from quote 9: (9, 1)
(word_id, word_count) from quote 10: (10, 1)
(word_id, word_count) from quote 11: (11, 1)


So `itertools.chain.from_iterable` joins all tuples.

Using a `for` loop we use create a word_id entry in the empty dictionary and for each of the words corresponding to these id we sum all its occurrences in the corpus:

In [116]:
total_word_count = defaultdict(int)
for word_id, word_count in itertools.chain.from_iterable(corpus):
    total_word_count[word_id] += word_count
sorted_word_count = sorted(total_word_count.items(), key=lambda w: w[1], reverse=True) 
print(sorted_word_count)

[(2, 6), (23, 6), (12, 4), (15, 4), (16, 4), (20, 4), (0, 3), (5, 3), (8, 3), (19, 3), (1, 1), (3, 1), (4, 1), (6, 1), (7, 1), (9, 1), (10, 1), (11, 1), (13, 1), (14, 1), (17, 1), (18, 1), (21, 1), (22, 1), (24, 1)]


We end up with the number of times each word appears in the full corpus:

In [133]:
for word_id, word_count in sorted_word_count[:5]:
    print('Frequency of the term'+' "'+dictionary.get(word_id)+'"'+' is:', word_count)

Frequency of the term "mastering" is: 6
Frequency of the term "genius" is: 6
Frequency of the term "deeds" is: 4
Frequency of the term "dozing" is: 4
Frequency of the term "anticipate" is: 4
